In [ ]:
import os
import sys
import random

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import LineCollection

In [ ]:
sys.path.append('../../utils/')

In [ ]:
from dlc_helper import DLC_tracking
from features import *
from features_speed import *
from preprocess_dlc import *

In [ ]:
import ipywidgets.widgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
from scipy.signal import *

# Import results

In [ ]:
df_results_control = pd.read_hdf('../../../results/UMAP_HDBSCANclustering_withWV_31072023_1135.h5')

# Test: plot skeletons

In [ ]:
df_meta = pd.read_pickle('../../../data/amphioxus_metadata_final500.pickle')

In [ ]:
filepaths = df_meta['dlc_result_file']
len(filepaths)

In [ ]:
@interact_manual
def plot_skeleton_trajectory(fn=filepaths):
    
    # Read DLC data
    dlc_folder, dlc_filename = os.path.split(fn)
    dlc_obj = DLC_tracking(dlc_filename, dlc_folder)
    
    df_dorsal = dlc_obj.df_data.filter(regex='^(DT).*(x|y)$')
    df_dorsal_filt = df_dorsal[df_dorsal.isna().sum(axis=1) < 5]
    df_dorsal_x = df_dorsal_filt.filter(like='_x')
    df_dorsal_y = df_dorsal_filt.filter(like='_y')
    df_dorsal_interp_x = interpol_spatial(df_dorsal_x)
    df_dorsal_interp_y = interpol_spatial(df_dorsal_y)
    df_dorsal_x_fin = interpol_temporal(df_dorsal_interp_x)
    df_dorsal_y_fin = interpol_temporal(df_dorsal_interp_y)
    df_dorsal_fin = df_dorsal_x_fin.join(df_dorsal_y_fin)[df_dorsal.columns]
    dorsal_xy_vals = df_dorsal_fin.values.reshape((-1,10,2))

#     segments = np.hstack([dorsal_xy_vals[1:5:2000, :, :], dorsal_xy_vals[1:5:2001, :, :]])
    
    fig, axes = plt.subplots(1,2,figsize=(20,10))
    for i in np.arange(0,5000, 50):
        x_vals = dorsal_xy_vals[i, :,0]
        y_vals = dorsal_xy_vals[i, :,1]
        x_smooth = savgol_filter(x_vals, polyorder=3, window_length=7)
        y_smooth = savgol_filter(y_vals, polyorder=3, window_length=7)
        axes[0].plot(x_vals, y_vals, c='b')
        axes[1].plot(x_smooth, y_smooth, c='b')
    for ax in axes:
        ax.set_aspect('equal')
        

# 

In [ ]:
c_pal = sns.color_palette('tab10', 10)
c_dict = {i: c_pal[i+1] for i in [-1,0,1,2,3,4,5,6]}

In [ ]:
@interact_manual
def plot_skeletons_cluster(fn=filepaths):
    print(fn)
    
    # Read DLC data
    dlc_folder, dlc_filename = os.path.split(fn)
    dlc_obj = DLC_tracking(dlc_filename, dlc_folder)
    
    df_dorsal = dlc_obj.df_data.filter(regex='^(D).*(x|y)$')
    df_dorsal_filt = df_dorsal[df_dorsal.isna().sum(axis=1) < 5]
    df_dorsal_x = df_dorsal_filt.filter(like='_x')
    df_dorsal_y = df_dorsal_filt.filter(like='_y')
    df_dorsal_interp_x = interpol_spatial(df_dorsal_x)
    df_dorsal_interp_y = interpol_spatial(df_dorsal_y)
    df_dorsal_x_fin = interpol_temporal(df_dorsal_interp_x)
    df_dorsal_y_fin = interpol_temporal(df_dorsal_interp_y)
    df_dorsal_fin = df_dorsal_x_fin.join(df_dorsal_y_fin)[df_dorsal.columns]
    df_dorsal_fin = df_dorsal_fin.reindex_like(df_dorsal_filt)
    df_dorsal_fin['frame'] = dlc_obj.df_data.loc[df_dorsal_filt.index, 'frame']
 
    # Read clustering results
    df_result_fn = df_results_control[df_results_control['dlc_result_file'] == fn]
    
    # data from clustering
    df_cluster = df_dorsal_fin.merge(df_result_fn, on='frame', how='left')
    df_cluster = df_cluster.loc[df_cluster['hdbscan_wv_scaled'].notna()]
    clusters_ = [clus for clus in df_cluster['hdbscan_wv_scaled'].unique()]
    df_cluster['hue'] = [c_dict[clus] for clus in df_cluster['hdbscan_wv_scaled']]

    
    fig, axes = plt.subplots(4,2,figsize=(10,20), sharex=True, sharey=True)
    axes = axes.ravel()
    
    for i, (name, group) in enumerate(df_cluster.groupby('hdbscan_wv_scaled')):

        clus_ind = int(name)
        try:
            sample_skels = group.sample(n=1000)
            
        except ValueError as ve:
            total_samples = len(group.index)
            sample_skels = group.sample(n=total_samples)
            
        n_samples = 0
        for ind, row in sample_skels.iterrows():
            
            x_vals = row.filter(regex='^(D).*(x)$').values
            y_vals = row.filter(regex='^(D).*(y)$').values
            x_smooth = savgol_filter(x_vals, polyorder=3, window_length=13)
            y_smooth = savgol_filter(y_vals, polyorder=3, window_length=13)
            x_smooth_trans = x_smooth - x_smooth[0]
            y_smooth_trans = y_smooth - y_smooth[0]
            n_samples += 1
            axes[clus_ind].plot(x_smooth_trans, y_smooth_trans, c=row['hue'])
        
        axes[clus_ind].set_aspect('equal')
        axes[clus_ind].set_title(f'cluster_{name}_samples_{n_samples}')
        

In [ ]:
from features import *

In [ ]:
@interact_manual
def plot_skeletons_cluster(fn=filepaths):
    
    # Read DLC data
    dlc_folder, dlc_filename = os.path.split(fn)
    dlc_obj = DLC_tracking(dlc_filename, dlc_folder)
    
    # Interpolate missing datapoints (ventral) if less than 2 ventral points are missing!
    df_ventral = dlc_obj.df_data.filter(regex='^V.*(x|y)$')
    df_ventral_filt = df_ventral[df_ventral.isna().sum(axis=1) == 0]
    df_ventral_x = df_ventral_filt.filter(like='_x')
    df_ventral_y = df_ventral_filt.filter(like='_y')
    df_ventral_interp_x = interpol_spatial(df_ventral_x)
    df_ventral_interp_y = interpol_spatial(df_ventral_y)
    df_ventral_x_fin = interpol_temporal(df_ventral_interp_x)
    df_ventral_y_fin = interpol_temporal(df_ventral_interp_y)
    df_ventral_fin = df_ventral_x_fin.join(df_ventral_y_fin)[df_ventral.columns]
    df_ventral_fin = df_ventral_fin.reindex_like(df_ventral_filt)
    df_ventral_fin['frame'] = dlc_obj.df_data.loc[df_ventral_filt.index, 'frame']
 
    # Read clustering results
    df_result_fn = df_results_control[df_results_control['dlc_result_file'] == fn]
    
    # data from clustering
    df_cluster = df_ventral_fin.merge(df_result_fn, on='frame', how='left')
    df_cluster = df_cluster.loc[df_cluster['hdbscan_wv_scaled'].notna()]
    clusters_ = [clus for clus in df_cluster['hdbscan_wv_scaled'].unique()]
    df_cluster['hue'] = [c_dict[clus] for clus in df_cluster['hdbscan_wv_scaled']]

    
    fig, axes = plt.subplots(4,2,figsize=(10,20), sharex=True, sharey=True)
    axes = axes.ravel()
    
    for i, (name, group) in enumerate(df_cluster.groupby('hdbscan_wv_scaled')):

        clus_ind = int(name)
        try:
            sample_skels = group.sample(n=1000)
            
        except ValueError as ve:
            total_samples = len(group.index)
            sample_skels = group.sample(n=total_samples)
            
        n_samples = 0
        
        for ind, row in sample_skels.iterrows():
            
            x_vals = row.filter(regex='^(V).*(x)$').values
            y_vals = row.filter(regex='^(V).*(y)$').values
            x_smooth = savgol_filter(x_vals, polyorder=3, window_length=9)
            y_smooth = savgol_filter(y_vals, polyorder=3, window_length=9)
            x_smooth_trans = x_smooth - x_smooth[1]
            y_smooth_trans = y_smooth - y_smooth[1]
            xy_vals = np.zeros((1,x_vals.shape[0],2))
            xy_vals[:,:,0] = x_smooth_trans
            xy_vals[:,:,1] = y_smooth_trans
            v_len = get_length(xy_vals, len_type='sum_point_to_point')
            if v_len < 100:
                n_samples += 1
                axes[clus_ind].plot(x_smooth_trans, y_smooth_trans, c=row['hue'])
        
        axes[clus_ind].set_aspect('equal')
        axes[clus_ind].set_title(f'cluster_{name}_samples_{n_samples}')
        